## Final project

In [1]:
import facebook_scraper as fs
import pandas as pd
import datetime
import numpy as np
import os

/home/vnn04/miniconda3/envs/ml-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
FANPAGE_LINK = "sieuanhhungMarvel"
current_dir = os.getcwd()
FOLDER_PATH = current_dir + "/Data/"
print(FOLDER_PATH)

/home/vnn04/Documents/facebook_scraper_page/Data/


#### Nhập tài khoản và mật khẩu facebook

In [3]:
user_email = "tomnguyen02102022@gmail.com"
user_password = "34md506647"

In [14]:
day_start = datetime.datetime(2023, 11, 11, 0, 0, 0)
day_end = datetime.datetime(2023, 9, 11, 0, 0, 0)

#### Lấy thông tin của các bài viết trong 2 tháng vừa qua (từ 11/9/2023 - 11/11/2023)

In [18]:
post_list = []

for post in fs.get_posts(FANPAGE_LINK,
                    options={"comment":False, "reactions":False, "allow_extra_requests":False},
                    extra_info=False, pages=1000, credentials=(user_email, user_password)):
    if post['time'] > day_start:
        continue
    else:
        if post['time'] <= day_end:
            break
        print(post)
        post_list.append({'post_id':post['post_id'], 'time':post['time'], 'text':post['text']})


{'post_id': '730004149170923', 'text': 'Chúng ta hãy cùng nhau một lần nữa nhìn lại hình ảnh này - khoảnh khắc quyết định vận mệnh và lịch sử của toàn bộ Vũ Trụ:', 'post_text': 'Chúng ta hãy cùng nhau một lần nữa nhìn lại hình ảnh này - khoảnh khắc quyết định vận mệnh và lịch sử của toàn bộ Vũ Trụ:', 'shared_text': '', 'original_text': None, 'time': datetime.datetime(2023, 11, 10, 23, 35, 6), 'timestamp': 1699634106, 'image': None, 'image_lowquality': 'https://scontent.fhan4-2.fna.fbcdn.net/v/t39.30808-6/399653794_730003749170963_2531783567474086640_n.jpg?stp=cp0_dst-jpg_e15_q65_s320x320&_nc_cat=111&ccb=1-7&_nc_sid=5f2048&efg=eyJpIjoidCJ9&_nc_ohc=LQkij9FXbVIAX-Qg2uF&_nc_ht=scontent.fhan4-2.fna&oh=00_AfDCfJUwgxHytEPWQ4Y82hixPLoAYQ5rMoAukad5mpnc7g&oe=6554EE31', 'images': None, 'images_description': None, 'images_lowquality': ['https://scontent.fhan4-2.fna.fbcdn.net/v/t39.30808-6/399653794_730003749170963_2531783567474086640_n.jpg?stp=cp0_dst-jpg_e15_q65_s320x320&_nc_cat=111&ccb=1-7&_nc_s

In [19]:
df = pd.DataFrame(post_list)
df.head()

,post_id,time,text
0,730004149170923,2023-11-10 23:35:06,Chúng ta hãy cùng nhau một lần nữa nhìn lại hì...
1,729861169185221,2023-11-10 18:27:51,"""Ta là Loki của Asgard. Và ta gánh vác trên mì..."
2,729701689201169,2023-11-10 11:35:33,ĐM TẬP CUỐI TUYỆT VỜI🤯😱💯!\nGLORIOUS PURPOSE!!!...
3,729692509202087,2023-11-10 11:04:08,Tất cả các phim trong năm 2024 của MCU đều sẽ ...
4,729666115871393,2023-11-10 09:48:32,Theo thông báo mới của Disney thì năm sau chỉ ...


In [20]:
print(len(post_list))

199


#### Lưu danh sách thông tin cơ bản của các post vào file sieuanhhungMarvel.csv

In [24]:
import csv 
path = FOLDER_PATH + FANPAGE_LINK + "_post.csv"

with open(path, mode='w', newline='') as file:
    writer = csv.writer(file)
    for post_id in post_list:
        writer.writerow([post_id])


#### Đọc 2000 id từ file sieuanhhungMarvel_post_id.csv

In [5]:
path = FOLDER_PATH + FANPAGE_LINK + "_post_id.csv"
df_post_id = pd.read_csv(path, names=['post_id'])
df_post_id.head(5)

,post_id
0,728483025989702
1,728409392663732
2,728332969338041
3,728155199355818
4,728132902691381
